# secrets

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime as dt
import pandas as pd
import numpy as np
import pprint
import boto3
import json
import time
import math
import sys
import os

# os.chdir(path = '/home/steven/Documents/reddit-streaming/')
pp = pprint.PrettyPrinter(indent = 1)
base = os.getcwd()
# os.chdir(os.path.join(base, "reddit"))
# base = os.getcwd()
print("imported modules, base: {}.".format(base))

imported modules, base: /opt/workspace/redditStreaming/src/main/python.


## read creds json

In [3]:
with open("creds.json", "r") as f:
    creds = json.load(f)
    f.close()

# pp.pprint(creds)

In [ ]:
test = {'aws_client': '',
 'aws_secret': '',
 'client_id': '',
 'password': '',
 'secret_id': '',
 'subreddit': 'technology',
 'user': 'SJH823'}

pp.pprint(test)

## spark

In [ ]:
spark_host = "spark-master" 
kafka_host = "kafka" 
subreddit = "cosmos"
aws_client = creds["aws_client"]
aws_secret = creds["aws_secret"]
spark_version = "3.4.0"
hadoop_version = "3.3.4"
delta_version = "1.2.1"
postgres_version = "9.4.1212"

spark = SparkSession.builder.appName(subreddit) \
    .master("spark://{}:7077".format(spark_host)) \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .config("spark.scheduler.mode", "FAIR") \
    .config("spark.scheduler.allocation.file", "file:///opt/workspace/redditStreaming/fairscheduler.xml") \
    .config("spark.executor.memory", "4096m") \
    .config("spark.executor.cores", "4") \
    .config("spark.streaming.concurrentJobs", "4") \
    .config("spark.local.dir", "/opt/workspace/tmp/driver/{}/".format(subreddit)) \
    .config("spark.worker.dir", "/opt/workspace/tmp/executor/{}/".format(subreddit)) \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "file:///opt/workspace/events/{}/".format(subreddit)) \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .config("spark.jars.packages", f"org.apache.spark:spark-sql-kafka-0-10_2.12:{spark_version},org.apache.hadoop:hadoop-common:{hadoop_version},org.apache.hadoop:hadoop-aws:{hadoop_version},org.apache.hadoop:hadoop-client:{hadoop_version},io.delta:delta-core_2.12:{delta_version},postgresql:postgresql-2.12:{postgres_version}") \
    .config("spark.hadoop.fs.s3a.access.key", aws_client) \
    .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .config('spark.hadoop.fs.s3a.buffer.dir', '/opt/workspace/tmp/blocks') \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    .enableHiveSupport() \
    .getOrCreate()

# sc = spark.SparkContext
# sc.setLogLevel('WARN')
print("created spark successfully.")

try:
    spark = SparkSession.builder.appName(subreddit).getOrCreate()
    # sc = spark.sparkContext
    # sc.setLogLevel('WARN')
    print("Created Spark successfully.")
except Exception as e:
    print(f"Failed to create Spark session: {str(e)}")
    raise

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
postgresql#postgresql-2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d86e8a86-746d-4b56-8d47-83694494a28d;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.0 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.9.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.

RuntimeError: Java gateway process exited before sending its port number